In [2]:
!pip install boto3
!pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=027afd19602ab10fec1a35e8028aff22525f4f435e657dcf3876b13a6505e0c7
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
import pickle
import boto3
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#instalar tambien setuptools

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc)

In [5]:
#Vale
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
#sc = spark.sparkContext         # Since Spark 2.0 'spark' is a SparkSession object that is by default created upfront and available in Spark shell, you need to explicitly create SparkSession object by using builder
spark = SparkSession.builder.getOrCreate()
#sc = SparkContext().getOrCreate()
sc = SparkContext._active_spark_context #devuelve la instancia existente
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc,spark)

In [9]:
# These paths should be changed to wherever you want to save the general data and where you want to save
# iteration specific data
base_save_path = "/content/drive/MyDrive/ProyectoRanking/parsinginstitoa/002_Model/"
iteration_save_path = "/content/drive/MyDrive/ProyectoRanking/parsinginstitoa/002_Model/institutional_affiliation_classification/"
rutaDatos = "/content/drive/MyDrive/ProyectoRanking/parsinginstitoa/Datos"

### Getting all data (From saved OpenAlex DB snapshot)

In [10]:
institutions = spark.read.parquet(f"{rutaDatos}/OA_static_institutions_single_file.parquet") \
    .filter(F.col('ror_id')!='')

In [11]:
institutions.cache().count()

331

In [12]:
df_affil = pd.read_csv(f"{rutaDatos}/Insumos_M1/file_parts_20231201_arg.csv")
df_affil = df_affil.rename(columns={'affiliation_ids': 'affiliation_id'})
df_affil.to_parquet(f"{rutaDatos}/static_affiliations.parquet")

In [14]:
print(df_affil.shape)
print(df_affil.columns)
df_affil.head(1)

(464282, 3)
Index(['unique_string_id', 'affiliation_id', 'original_affiliation'], dtype='object')


,unique_string_id,affiliation_id,original_affiliation
0,4.724468e+11,4.210103e+09,"Multiple Sclerosis University Center, J.M. Ram..."


In [15]:
affiliations = spark.read.parquet(f"{rutaDatos}/static_affiliations.parquet")

In [16]:
affiliations.cache().count()

464282

In [17]:
affiliations.head(1)

[Row(unique_string_id=472446798877.0, affiliation_id=4210103251.0, original_affiliation='Multiple Sclerosis University Center, J.M. Ramos Mejía Hospital, Buenos Aires, Argentina')]

#### Getting ROR aff strings

In [18]:
dedup_affs = affiliations.select(F.trim(F.col('original_affiliation')).alias('original_affiliation'), 'affiliation_id')\
.filter(F.col('original_affiliation').isNotNull())\
.filter(F.col('original_affiliation')!='')\
.withColumn('aff_len', F.length(F.col('original_affiliation')))\
.filter(F.col('aff_len')>2)\
.groupby(['original_affiliation','affiliation_id']) \
.agg(F.count(F.col('affiliation_id')).alias('aff_string_counts'))

In [19]:
print(dedup_affs.columns)
dedup_affs.cache().count()

['original_affiliation', 'affiliation_id', 'aff_string_counts']


463812

In [20]:
ror_data = spark.read.parquet(f"{rutaDatos}/ror_strings.parquet").select('original_affiliation','affiliation_id')

In [21]:
ror_data.cache().count()

45981

### Gathering training data

Since we are looking at all institutions, we need to up-sample the institutions that don't have many affiliation strings and down-sample the institutions that have large numbers of strings. There was a balance here that needed to be acheived. The more samples that are taken for each institution, the more overall training data we will have and the longer our model will take to train. However, more samples also means more ways of an institution showing up in an affiliation string. The number of samples was set to 50 as it was determined this was a good optimization point based on affiliation string count distribution and time it would take to train the model. However, unlike in V1 where we tried to keep all institutions at 50, for V2 we gave additional samples for institutions with more strings available. Specifically, we allowed those institutions to have up to 25 additional strings, for a total of 75.

In [22]:
num_samples_to_get = 50
dedup_affs.head(1)

[Row(original_affiliation='Institute of Polymer Technology and Nanotechnology, Facultad de Arquitectura Diseño y Urbanismo, Universidad de Buenos Aires-CONICET, Ciudad de Buenos Aires, Argentina.', affiliation_id=2.4354313151201028e+16, aff_string_counts=1)]

In [25]:
w1 = Window.partitionBy('affiliation_id')

filled_affiliations = dedup_affs \
    .join(ror_data.select('affiliation_id'), how='inner', on='affiliation_id') \
    .select('original_affiliation','affiliation_id') \
    .union(ror_data.select('original_affiliation','affiliation_id')) \
    .filter(~F.col('affiliation_id').isNull()) \
    .dropDuplicates() \
    .withColumn('random_prob', F.rand(seed=20)) \
    .withColumn('id_count', F.count(F.col('affiliation_id')).over(w1)) \
    .withColumn('scaled_count', F.lit(1)-((F.col('id_count') - F.lit(num_samples_to_get))/(F.lit(3500) - F.lit(num_samples_to_get)))) \
    .withColumn('final_prob', F.col('random_prob')*F.col('scaled_count'))

#3500000

In [26]:
filled_affiliations.select('affiliation_id').distinct().count()

331

In [27]:
less_than = filled_affiliations.dropDuplicates(subset=['affiliation_id']).filter(F.col('id_count') < num_samples_to_get).toPandas()
less_than.shape

(159, 6)

In [28]:
less_than.sample(10)

,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob
25,PLAPIQUI Bahia Blanca Argentina,https://openalex.org/I4210097725,0.518178,26,1.006957,0.521782
113,Instituto de Investigacion Medica Mercedes y M...,https://openalex.org/I4210145890,0.403877,39,1.003188,0.405164
53,"Ministerio de Salud, Buenos Aires",https://openalex.org/I4210110275,0.288504,26,1.006957,0.290511
32,International Network of Civil Liberties Organ...,https://openalex.org/I4210101536,0.838179,26,1.006957,0.844010
35,"INTEC, Argentina",https://openalex.org/I4210102755,0.719808,14,1.010435,0.727319
142,"Junior Achievement, Rosario, Santa Fe, Argentina",https://openalex.org/I4210158077,0.471433,26,1.006957,0.474712
62,INQUINOA Tucuman,https://openalex.org/I4210116650,0.493447,26,1.006957,0.496880
79,"Wikimedia Argentina, Buenos Aires",https://openalex.org/I4210125659,0.973077,13,1.010725,0.983513
157,IByME Buenos Aires,https://openalex.org/I71650599,0.072049,39,1.003188,0.072279
56,CIQUIBIC Cordoba,https://openalex.org/I4210112829,0.468503,25,1.007246,0.471898


In [29]:
temp_df_list = []
for aff_id in less_than['affiliation_id'].unique():
    temp_df = less_than[less_than['affiliation_id']==aff_id].copy()
    help_df = temp_df.sample(num_samples_to_get - temp_df.shape[0], replace=True)
    temp_df_list.append(pd.concat([temp_df, help_df], axis=0))
less_than_df = pd.concat(temp_df_list, axis=0)

In [30]:
less_than_df.shape

(7950, 6)

In [31]:
# only install fsspec and s3fs
less_than_df[['original_affiliation', 'affiliation_id']].to_parquet(f"{iteration_save_path}lower_than_{num_samples_to_get}.parquet")

In [32]:
w1 = Window.partitionBy('affiliation_id').orderBy('random_prob')

more_than = filled_affiliations.filter(F.col('id_count') >= num_samples_to_get) \
.withColumn('row_number', F.row_number().over(w1)) \
.filter(F.col('row_number') <= num_samples_to_get+25)

In [33]:
more_than.cache().count()

12656

In [34]:
more_than.select('original_affiliation', 'affiliation_id').coalesce(1).write.mode('overwrite').parquet(f"{iteration_save_path}more_than_{num_samples_to_get}.parquet")